# Binary Search Tree Check

## Problem Statement

Given a binary tree, check whether it’s a binary search tree or not.

**Again, no solution cell, just worry about your code making sense logically. Hint: Think about tree traversals.**

This is a classic interview problem, so feel free to just Google search "Validate BST" for more information on this problem!


In [18]:
class node(object):
    def __init__(self, key, value, lc=None, rc=None):
        self.key = key
        self.value = value
        self.left_child = lc
        self.right_child = rc

In [ ]:
# some functions

## Solution


In [ ]:

def get_AllValue(tree_node, output=None):
    '''use inorder to traverse tree, due to the feature of bst'''

    if output == None:
        output = []

    # Recursion:
    #   - base case1: if current node is none, do nothing
    #   - base case2: if current node is not none, for the key, append key
    if tree_node:
        # recursively get value from left_child
        get_AllValue(tree_node.left_child, output)
        output.append(tree_node.key)
        get_AllValue(tree_node.right_child, output)
    # print(output)
    return output

#  verify


def verify(tree_node):
    '''get all value from tree, compared values with sorted values'''
    value_list = get_AllValue(tree_node)
    return value_list == sorted(value_list)


node01 = node(1, "one")
node03 = node(3, "three")
node02 = node(2, "two", node01, node03)

print(verify(node02))       # true

node04 = node(4, "four")
node01.left_child = node04

print(verify(node02))       # False

True
False


Another classic solution is to keep track of the minimum and maximum values a node can take. And at each node we will check whether its value is between the min and max values it’s allowed to take. The root can take any value between negative infinity and positive infinity. At any node, its left child should be smaller than or equal than its own value, and similarly the right child should be larger than or equal to. So during recursion, we send the current value as the new max to our left child and send the min as it is without changing. And to the right child, we send the current value as the new min and send the max without changing.

- 思路:
  - bst 恒有 max(left descendent) < parent < min(left descendent)
  - 对 each node 递归验证
  - base case:
    - current = none, 返回 true
    - current = leaf, 返回验证结果


In [38]:
class Node:
    def __init__(self, k, val):
        self.key = k
        self.value = val
        self.left = None
        self.right = None


def tree_max(node):
    if not node:
        return float("-inf")
    maxleft = tree_max(node.left)
    maxright = tree_max(node.right)
    return max(node.key, maxleft, maxright)


def tree_min(node):
    if not node:
        return float("inf")
    minleft = tree_min(node.left)
    minright = tree_min(node.right)
    return min(node.key, minleft, minright)


def verify(node):
    if not node:
        return True
    if (tree_max(node.left) <= node.key <= tree_min(node.right) and
            verify(node.left) and verify(node.right)):
        return True
    else:
        return False


root = Node(10, "Hello")
root.left = Node(5, "Five")
root.right = Node(30, "Thirty")

print(verify(root))  # prints True, since this tree is valid

root = Node(10, "Ten")
root.right = Node(20, "Twenty")
root.left = Node(5, "Five")
root.left.right = Node(15, "Fifteen")

print(verify(root))  # prints False, since 15 is to the left of 10

True
False


---
## Tree Level Order Print

Given a binary tree of integers, print it in level order. The output will contain space between the numbers in the same level, and new line between different levels. For example, if the tree is:
---

![title](./pic/tree_print.png)

---

The output should be:

    1
    2 3
    4 5 6


### Solution

- It won’t be practical to solve this problem using recursion, **because recursion is similar to depth first search,** but what we need here is breadth first search. So we will **use a queue as we did previously in breadth first search**.

  - First, we’ll push the root node into the queue.
  - Then we start a while loop with the condition queue not being empty.
  - Then, at each iteration we pop a node from the beginning of the queue and push its children to the end of the queue. Once we pop a node we print its value and space.

- To print the new line in correct place we should count the number of nodes at each level. We will have 2 counts, namely current level count and next level count.

  - **Current level count** indicates how many nodes should be printed at this level before printing a new line.
  - We decrement it every time we pop an element from the queue and print it.
  - Once the current level count reaches zero we **print a new line**.
  - **Next level count** contains the number of nodes in the next level, which will become the current level count after printing a new line.
  - We count the number of nodes in the next level by counting the number of children of the nodes in the current level.

- 思路:
  - 使用 queue 的 FIFO
    - 在 python 中使用 collections.deque
      - 使用 append 实现 in
      - 使用 popleft 实现 out
    - 对 queue 循环, 如果 node 有 left/right, queue 增加;
    - 利用 leaf 没有 left/right, 所以循环直到最后一行没有时, queue 不再增加.
  - 难点在于如何换行
    - 使用 current count 对本行进行计数, 因为是 tree 结构, 只有一个 root, 所以初始值是 1.
    - 每个 node 打印后, 递减直到 0, 即换行
    - 使用 next count 对 children 进行计数, 因为无法确定当前是否 leaf,所以 next count 初始是 0
    - 每次循环结束时, current 和 next 互换.


In [1]:
class Node:
    def __init__(self, val=None):
        self.left = None
        self.right = None
        self.val = val

In [14]:
import collections


def levelOrderPrint(tree):
    if not tree:
        return
    nodes = collections.deque([tree])
    currentCount, nextCount = 1, 0
    while len(nodes) != 0:
        currentNode = nodes.popleft()
        currentCount -= 1
        print(currentNode.val, end="")
        if currentNode.left:
            nodes.append(currentNode.left)
            nextCount += 1
        if currentNode.right:
            nodes.append(currentNode.right)
            nextCount += 1
        if currentCount == 0:
            # finished printing current level
            print('\n', end='')
            currentCount, nextCount = nextCount, currentCount

In [15]:
root = Node(1)

root.left = Node(2)
root.right = Node(3)

levelOrderPrint(root)

1
23


---
## Trim a Binary Search Tree

- Given the root of a binary search tree and 2 numbers min and max, trim the tree such that all the numbers in the new tree are between min and max (inclusive). The resulting tree should still be a valid binary search tree. So, if we get this tree as input:
---

![title](./pic/bst1.png)

---

and we’re given **min value as 5** and **max value as 13**, then the resulting binary search tree should be:

---

![title](./pic/bst_trim.png)

---

We should remove all the nodes whose value is not between min and max.

---

** Feel free to reference the lecture on Binary Search Tree for the node class, but what it more important here is the logic of your function. In which case your function should be in the form:**


In [ ]:
class Node:
    def __init__(self, val=None):
        self.left = None
        self.right = None
        self.val = val


def trimBST(tree, minVal, maxVal):

    print(tree.left)  # LeftChild
    print(tree.right)  # Right Child
    print(tree.val)  # Node's value

    pass

# Use tree.left , tree.right , and tree.val as your methods to call


# def traverse(root, output=None):
#     if output == None:
#         output = []

#     if root.left != None:
#         traverse(root.left, output)

#     output.append(root.val)

#     if root.right != None:
#         traverse(root.right, output)

### Solution

- We can do this by performing a **post-order traversal** of the tree.

  - We first process the left children, then right children, and finally the node itself.
  - So we form the new tree bottom up, starting from the leaves towards the root.
  - As a result while processing the node itself, both its left and right subtrees are valid trimmed binary search trees (may be NULL as well).

- At each node we’ll return a reference based on its value, which will then be assigned to its parent’s left or right child pointer, depending on whether the current node is left or right child of the parent.
  - If current node’s value is **between min and max** (min<=node<=max) then there’s **no action** need to be taken, so we return the reference to the node itself.
  - If current node’s value is **less than min**, then we return the reference to its right subtree, and discard the left subtree. Because if a node’s value is less than min, then **its left children are definitely less than min** since this is a binary search tree. But its right children may or may not be less than min we can’t be sure, so we return the reference to it.
  - Since we’re performing **bottom-up post-order traversal**, its **right subtree** is already a trimmed valid binary search tree (possibly NULL), and left subtree is definitely NULL because those nodes were surely less than min and they were eliminated during the post-order traversal.
- Remember that **in post-order traversal we first process all the children of a node, and then finally the node itself.**

- Similar situation occurs when node’s value is greater than max, we now return the reference to its left subtree. Because if a node’s value is greater than max, then its right children are definitely greater than max. But its left children may or may not be greater than max. So we discard the right subtree and return the reference to the already valid left subtree.

- The complexity of this algorithm is `O(N)`, where N is the number of nodes in the tree. Because we basically perform a post-order traversal of the tree, visiting each and every node one. This is optimal because we should visit every node at least once. This is a very elegant question that demonstrates the effectiveness of recursion in trees.


In [57]:
class Node:
    def __init__(self, val=None, left=None, right=None):
        self.left = left
        self.right = right
        self.val = val


def trimBST(tree, minVal, maxVal):

    if not tree:
        return

    tree.left = trimBST(tree.left, minVal, maxVal)
    tree.right = trimBST(tree.right, minVal, maxVal)

    # 在范围时, left, right 不变
    if minVal <= tree.val <= maxVal:
        return tree

    # 小于, left = right, left尝试向范围靠拢; 下一次递归时, 如果在范围则不变; 否则继续靠拢
    # 如果right = none, 则收敛
    if tree.val < minVal:
        return tree.right

    # 同理, 大约, right=left, right尝试向范围靠拢;
    # 如果left = none, 则收敛
    if tree.val > maxVal:
        return tree.left

- 思路总结:
  - 使用 post-order, 效果先递归到 leaf.
    - 代码中是先 left 后 right, 所以向左叶开始递归然后开始向上收敛
  - 因为 bst 的定义, 所以外<内, 左<右. 递归一定保证在 leaf 收敛后的树还是 bst
  - 当 node 在范围内时, node 不变
  - 当 node < min 时, 返回 right, 则递归向范围靠拢, 直到:
    - 递归回到范围
    - node=none, 则收敛
  - 当 node > max 时, 返回 left, 则递归向返回靠拢, 直到:
    - 递归回到范围
    - node=none, 则收敛
